# Set up and global variables

In [ ]:
from pathlib import Path
from copy import deepcopy

import os
import pickle
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np

from IPython.display import display, HTML
from tqdm import tqdm
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc
from sklearn.pipeline import Pipeline
from scipy.stats import wilcoxon
from sklearn.calibration import calibration_curve

from src.prioritization import *

RETRAIN_MODELS = True

In [ ]:
os.environ["CONFIG_ENV"] = "debug"
if False:
    os.environ["CONFIG_ENV"] = "production"

from config import load_config
config = load_config()

RESOLUTION = config['DEFAULTS']['resolution']
SEED = config['DEFAULTS']['random_seed']

# input data
BENCHMARK_PATH = config['PATHS']['benchmark_dataset']
STORAGE_PATH = config['PATHS']['storage']

# output data
IMAGE_DIR = config['PATHS']['images'] / 'modelling'
FINAL_MODEL_PATH = BENCHMARK_PATH / "final_teacher_model.pkl"
FINAL_FEATURES_PATH = BENCHMARK_PATH / "final_selected_features.pkl"
FINAL_HEURISTIC_MODEL_PATH = BENCHMARK_PATH / "final_heuristic_model.pkl"
FINAL_HEURISTIC_FEATURES_PATH = BENCHMARK_PATH / "final_heuristic_selected_features.pkl"
BENCHMARK_CACHE_PATH = BENCHMARK_PATH / "benchmark_cache"

os.makedirs(IMAGE_DIR, exist_ok=True)
os.makedirs(BENCHMARK_CACHE_PATH, exist_ok=True)

***

# Loading data

In [ ]:
items = pd.read_csv(STORAGE_PATH / 'items.csv', index_col=0)
defects = pd.read_csv(STORAGE_PATH / f'defects.csv', index_col=0)

df = pd.read_csv(BENCHMARK_PATH / 'benchmark_dataset.csv')

In [ ]:
left_discrete_features = [col for col in df.columns if col.endswith('(Left Discrete)')]
right_discrete_features = [col for col in df.columns if col.endswith('(Right Discrete)')]
left_continuous_features = [col for col in df.columns if col.endswith('(Left Continuous)')]
right_continuous_features = [col for col in df.columns if col.endswith('(Right Continuous)')]

if any(map(lambda x: len(x) == 0, [left_discrete_features, right_discrete_features, left_continuous_features, right_continuous_features])):
    raise ValueError('Some of the feature sets are empty')

***

# Feature engineering

In [ ]:
def remove_suffix(col: str) -> str:
    """Return base name before the first ' (' occurrence; fall back to original."""
    if not isinstance(col, str):
        return col
    idx = col.find(" (")
    return col[:idx] if idx != -1 else col

In [ ]:
left_discrete_values = df[left_discrete_features].rename(columns=remove_suffix)
right_discrete_values = df[right_discrete_features].rename(columns=remove_suffix)

left_continuous_values = df[left_continuous_features].rename(columns=remove_suffix)
right_continuous_values = df[right_continuous_features].rename(columns=remove_suffix)

## Difference features

In [ ]:
discrete_diff = left_discrete_values - right_discrete_values
discrete_diff = discrete_diff.add_suffix(' (Discrete Diff)')

continuous_diff = left_continuous_values - right_continuous_values
continuous_diff = continuous_diff.add_suffix(' (Continuous Diff)')

## Binary flags

In [ ]:
discrete_is_larger = left_discrete_values > right_discrete_values
discrete_is_larger = discrete_is_larger.add_suffix(' (Discrete >)')

continuous_is_larger = left_continuous_values > right_continuous_values
continuous_is_larger = continuous_is_larger.add_suffix(' (Continuous >)')

In [ ]:
left_is_extreme_max = left_discrete_values == 5
left_is_extreme_max = left_is_extreme_max.add_suffix(' (Left Max)')
left_is_extreme_min = left_discrete_values == 1
left_is_extreme_min = left_is_extreme_min.add_suffix(' (Left Min)')

## Item and defect metadata

In [ ]:
left_type = defects['defect type'].loc[df['left']].reset_index(drop=True).rename('left')
right_type = defects['defect type'].loc[df['right']].reset_index(drop=True).rename('right')

item_topic = items['topic'].loc[df['item']].reset_index(drop=True).rename('item')

In [ ]:
metadata_encoder = OneHotEncoder()

metadata = metadata_encoder.fit_transform(pd.concat([
    left_type,
    right_type,
    item_topic
], axis=1))

metadata = pd.DataFrame(metadata.toarray(), columns=metadata_encoder.get_feature_names_out())

# Feature groups and combined dataframe

In [ ]:
engineered_df = pd.concat([
    df[left_discrete_features],
    df[right_discrete_features],
    discrete_diff,
    df[left_continuous_features],
    df[right_continuous_features],
    continuous_diff,
    discrete_is_larger,
    continuous_is_larger,
    left_is_extreme_max,
    left_is_extreme_min,
    metadata,
], axis=1)

print("Final engineered dataframe shape:", engineered_df.shape)

In [ ]:
feature_groups = {
    "Left Discrete": left_discrete_features,
    "Right Discrete": right_discrete_features,
    "Discrete Diff": discrete_diff.columns.tolist(),
    "Left+Right Continuous": left_continuous_features + right_continuous_features,
    "Continuous Diff": continuous_diff.columns.tolist(),
    "Derived Rules": discrete_is_larger.columns.tolist() + continuous_is_larger.columns.tolist(),
    "Additional Rules": left_is_extreme_max.columns.tolist() + left_is_extreme_min.columns.tolist(),
    "Metadata": metadata.columns.tolist(),
    "All Features": engineered_df.columns.tolist()
}

***

# Experiment

In [ ]:
y = df['left won']

sample_weights = df.loc[engineered_df.index]['weight']
groups = df['submission id']
logo = LeaveOneGroupOut()

folds = list(enumerate(logo.split(engineered_df, y, groups)))

## Single feature models

In [ ]:
base_model = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", LogisticRegression(max_iter=5000, solver="lbfgs", random_state=SEED))
])

In [ ]:
heuristic_feature_names = list(map(remove_suffix, left_discrete_features))
X_full = engineered_df[feature_groups['Left Discrete'] + feature_groups['Discrete Diff']]

if RETRAIN_MODELS:
    single_feature_results = []
    single_feature_fold_predictions = {}

    for fold_idx, (train_idx, test_idx) in tqdm(folds, desc="Iterating over folds", total=groups.nunique()):
        X_train_full, X_test_full = engineered_df.iloc[train_idx], engineered_df.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        train_weights, test_weights = sample_weights.iloc[train_idx], sample_weights.iloc[test_idx]

        # Single-feature models
        for value_type in ['Discrete', 'Continuous']:
            for feature_name in heuristic_feature_names:
                # Collect data
                cols = [f'{feature_name} (Left {value_type})', f'{feature_name} ({value_type} Diff)']
                X_train = X_train_full[cols]
                X_test = X_test_full[cols]

                # Train and predict
                model = deepcopy(base_model)

                # Pass sample weights to the pipeline
                if isinstance(model, Pipeline):
                    model.fit(X_train, y_train, clf__sample_weight=train_weights)
                else:
                    model.fit(X_train, y_train, sample_weight=train_weights)

                y_pred = model.predict(X_test)
                y_proba = model.predict_proba(X_test)[:, 1]

                # Evaluate
                acc = accuracy_score(y_test, y_pred, sample_weight=test_weights)
                try:
                    auc_score = roc_auc_score(y_test, y_proba, sample_weight=test_weights)
                except ValueError:
                    # y_test might only contain one class
                    auc_score = np.nan

                # Store results
                model_name = "LogisticRegression"
                group_name = f"{feature_name} - {value_type}"

                single_feature_results.append({
                    "fold": fold_idx,
                    "model": model_name,
                    "group": group_name,
                    "accuracy": acc,
                    "auc": auc_score
                })

                single_feature_fold_predictions[(fold_idx, model_name, group_name)] = pd.DataFrame({
                    "y_true": y_test.values,
                    "y_pred": y_pred,
                    "y_proba": y_proba
                }, index=y_test.index)

    single_feature_results = pd.DataFrame(single_feature_results)
    single_feature_results.to_csv(BENCHMARK_CACHE_PATH / 'single_feature_results.csv', index=False)
    pickle.dump(single_feature_fold_predictions, open(BENCHMARK_CACHE_PATH / 'single_feature_fold_predictions.pkl', 'wb'))
else:
    single_feature_results = pd.read_csv(BENCHMARK_CACHE_PATH / 'single_feature_results.csv')
    single_feature_fold_predictions = pickle.load(open(BENCHMARK_CACHE_PATH / 'single_feature_fold_predictions.pkl', 'rb'))

In [ ]:
# Aggregate by model and feature group
single_feature_summary = single_feature_results.groupby(["model", "group"])[["accuracy", "auc"]].agg(["mean", "std"]).reset_index()
# flatten multiindex
single_feature_summary.columns = [" ".join(col).strip() for col in single_feature_summary.columns.values]
single_feature_summary['ranking_stability_score'] = single_feature_summary['auc mean'] - 0.5 * single_feature_summary['auc std']
single_feature_summary.sort_values('ranking_stability_score', ascending=False, inplace=True)
display(single_feature_summary)

## Complex models

In [ ]:
models = {
    "Random Forest": RandomForestClassifier(
        random_state=SEED,
        max_depth=3
    ),

    "Gradient Boosting": GradientBoostingClassifier(
        random_state=SEED,
        max_depth=3
    ),

    "Logistic Regression": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            random_state=SEED,
            max_iter=5000,
            solver="lbfgs"
        ))
    ]),
    "LASSO Logistic": Pipeline([
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(
            random_state=SEED,
            max_iter=5000,
            solver="liblinear",
            penalty="l1"
        ))
    ])
}

if RETRAIN_MODELS:
    results = []
    fold_predictions = {}

    for fold_idx, (train_idx, test_idx) in tqdm(enumerate(logo.split(engineered_df, y, groups=groups)), desc="Iterating over folds", total=groups.nunique()):
        # Select indices
        X_train_full, X_test_full = engineered_df.iloc[train_idx], engineered_df.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
        train_weights, test_weights = sample_weights.iloc[train_idx], sample_weights.iloc[test_idx]

        # Iterate over models and feature groups
        for model_name, model in models.items():
            for group_name, cols in feature_groups.items():
                # Select columns
                X_train = X_train_full[cols]
                X_test = X_test_full[cols]
        
                # Train and predict
                model = deepcopy(model)

                # Pass sample weights to the pipeline
                if isinstance(model, Pipeline):
                    model.fit(X_train, y_train, clf__sample_weight=train_weights)
                else:
                    model.fit(X_train, y_train, sample_weight=train_weights)
                    
                y_pred = model.predict(X_test)

                # Evaluate
                # fallback for models without predict_proba
                if hasattr(model, "predict_proba"):
                    y_proba = model.predict_proba(X_test)[:, 1]
                else:
                    y_proba = np.nan

                acc = accuracy_score(y_test, y_pred, sample_weight=test_weights)
                try:
                    auc_score = roc_auc_score(y_test, y_proba, sample_weight=test_weights)
                except ValueError:
                    # y_test might only contain one class
                    auc = np.nan

                # Store results
                results.append({
                    "fold": fold_idx,
                    "model": model_name,
                    "group": group_name,
                    "accuracy": acc,
                    "auc": auc_score
                })

                fold_predictions[(fold_idx, model_name, group_name)] = pd.DataFrame({
                    "y_true": y_test.values,
                    "y_pred": y_pred,
                    "y_proba": y_proba
                }, index=y_test.index)

    ablation_study_results = pd.DataFrame(results)
    ablation_study_results.to_csv(BENCHMARK_CACHE_PATH / 'ablation_study_results.csv', index=False)
    pickle.dump(fold_predictions, open(BENCHMARK_CACHE_PATH / 'ablation_study_fold_predictions.pkl', 'wb'))
else:
    ablation_study_results = pd.read_csv(BENCHMARK_CACHE_PATH / 'ablation_study_results.csv')
    fold_predictions = pickle.load(open(BENCHMARK_CACHE_PATH / 'ablation_study_fold_predictions.pkl', 'rb'))

In [ ]:
# Aggregate by model and feature group
ablation_study_summary = ablation_study_results.groupby(["model", "group"])[["accuracy", "auc"]].agg(["mean", "std"]).reset_index()
# flatten multiindex
ablation_study_summary.columns = [" ".join(col).strip() for col in ablation_study_summary.columns.values]
ablation_study_summary['ranking_stability_score'] = ablation_study_summary['auc mean'] - 0.5 * ablation_study_summary['auc std']
ablation_study_summary.sort_values('ranking_stability_score', ascending=False, inplace=True)
display(ablation_study_summary)


***

# Train the final model with the best performing parameters

In [ ]:
best_hyperparams = ablation_study_summary.iloc[0]
best_model_name, best_feature_group = best_hyperparams["model"], best_hyperparams["group"]
print(f"Selected model: {best_model_name}; feature group: {best_feature_group}")

In [ ]:
if RETRAIN_MODELS:
    best_model = models[best_model_name]
    selected_cols = feature_groups.get(best_feature_group)

    X_full = engineered_df[selected_cols]
    y_full = y

    best_model.fit(X_full, y_full)

    joblib.dump(best_model, FINAL_MODEL_PATH)
    joblib.dump(selected_cols, FINAL_FEATURES_PATH)
else:
    best_model = joblib.load(FINAL_MODEL_PATH)
    selected_cols = joblib.load(FINAL_FEATURES_PATH)

In [ ]:
best_single_heuristic_model = single_feature_summary.iloc[0]
best_single_heuristic_name, best_single_heuristic_group = best_single_heuristic_model["model"], best_single_heuristic_model["group"]
print(f"Selected feature group: {best_single_heuristic_group}")

In [ ]:
if RETRAIN_MODELS:
    best_single_feature_model = base_model
    feature_name, value_type = best_single_heuristic_group.split(" - ")
    selected_cols = [f'{feature_name} (Left {value_type})', f'{feature_name} ({value_type} Diff)']

    X_full = engineered_df[selected_cols]
    y_full = y

    best_single_feature_model.fit(X_full, y_full)

    joblib.dump(best_single_feature_model, FINAL_HEURISTIC_MODEL_PATH)
    joblib.dump(selected_cols, FINAL_HEURISTIC_FEATURES_PATH)
else:
    best_single_feature_model = joblib.load(FINAL_HEURISTIC_MODEL_PATH)
    selected_cols = joblib.load(FINAL_HEURISTIC_FEATURES_PATH)

***

# Results

## Single feature models

In [ ]:
plt.figure(figsize=(10,5))
sns.barplot(data=single_feature_summary, x="group", y="ranking_stability_score", hue="group")
plt.xticks(rotation=45, ha="right")
plt.title("Single feature models by stability score")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "single_feature_ranking_stability_score.png", dpi=RESOLUTION)
plt.close()

In [ ]:
single_feature_results

In [ ]:
plt.figure(figsize=(10, 5))
ax = sns.barplot(
    data=single_feature_results,
    x="group",
    y="accuracy",
    errorbar=("ci", 95),
    hue="group"
)

plt.xticks(rotation=90)
plt.title("Single-Feature Heuristics vs Full Model (Accuracy)")
plt.xlabel("Heuristic (Single Feature)")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "single_feature_accuracy.png", dpi=RESOLUTION)
plt.close()


### Wilcoxon vs best model

In [ ]:
# Add best performing model
combined_df = pd.concat([
    single_feature_results,
    ablation_study_results[(ablation_study_results["model"] == best_model_name) & (ablation_study_results["group"] == best_feature_group)]
], axis=0)

combined_df['model+group'] = combined_df.apply(lambda row: f"{row['group']} ({row['model']})", axis=1)

In [ ]:
pairwise_tests = []

best_model_df = ablation_study_results[(ablation_study_results["model"] == best_model_name) & (ablation_study_results["group"] == best_feature_group)]

for i, heuristic_group in enumerate(single_feature_results["group"].unique()):
    heuristic_df = single_feature_results[
        (single_feature_results['group'] == heuristic_group)
    ].sort_values("fold")

    stat, p = wilcoxon(heuristic_df["accuracy"], best_model_df["accuracy"])
    pairwise_tests.append({
        "model_1": heuristic_group,
        "model_2": f"{best_model_name} ({best_feature_group})",
        "wilcoxon_stat": stat,
        "p_value": p
    })

pd.DataFrame(pairwise_tests).sort_values("p_value")

## Top model-feature pairs

In [ ]:
top10 = ablation_study_summary.head(10)
plt.figure(figsize=(10,5))
sns.barplot(data=top10, x="group", y="ranking_stability_score", hue="model")
plt.xticks(rotation=45, ha="right")
plt.title("Top model+feature combinations by stability score")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "top_models.png", dpi=RESOLUTION)
plt.close()

## Model comparison

### Wilcoxon

In [ ]:
feature_group_to_compare = "All Features"

pairwise_tests = []
models_list = ablation_study_results["model"].unique()

for i, m1 in enumerate(models_list):
    for m2 in models_list[i+1:]:
        df1 = ablation_study_results[
            (ablation_study_results["model"] == m1) & 
            (ablation_study_results["group"] == feature_group_to_compare)
        ].sort_values("fold")

        df2 = ablation_study_results[
            (ablation_study_results["model"] == m2) & 
            (ablation_study_results["group"] == feature_group_to_compare)
        ].sort_values("fold")

        if len(df1) == len(df2) and len(df1) > 0:
            stat, p = wilcoxon(df1["accuracy"], df2["accuracy"])
            pairwise_tests.append({
                "model_1": m1,
                "model_2": m2,
                "wilcoxon_stat": stat,
                "p_value": p
            })

pd.DataFrame(pairwise_tests)

## Diagnostic plots

In [ ]:
plt.figure(figsize=(6, 6))

for model_name in ablation_study_results["model"].unique():
    y_true_all = []
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_true_all.extend(pred_df["y_true"])
            y_proba_all.extend(pred_df["y_proba"])

    fpr, tpr, _ = roc_curve(y_true_all, y_proba_all)
    roc_auc = auc(fpr, tpr)

    plt.plot(fpr, tpr, lw=2, label=f"{model_name} (AUC={roc_auc:.3f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve (Aggregated Across Folds)")
plt.legend()
plt.grid(True)
plt.savefig(IMAGE_DIR / "roc_curve.png", dpi=RESOLUTION)
plt.close()

In [ ]:
plt.figure(figsize=(6, 6))

for model_name in ablation_study_results["model"].unique():
    y_true_all = []
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_true_all.extend(pred_df["y_true"])
            y_proba_all.extend(pred_df["y_proba"])

    prob_true, prob_pred = calibration_curve(
        y_true_all, y_proba_all, n_bins=10, strategy="quantile"
    )

    plt.plot(prob_pred, prob_true, marker="o", label=model_name)

plt.plot([0, 1], [0, 1], "k--", label="Perfect")
plt.xlabel("Predicted Probability")
plt.ylabel("Empirical Win Rate")
plt.title("Calibration Plot")
plt.legend()
plt.grid(True)
plt.savefig(IMAGE_DIR / "calibration_curve.png", dpi=RESOLUTION)
plt.close()

In [ ]:
plt.figure(figsize=(8, 5))

for model_name in ablation_study_results["model"].unique():
    y_proba_all = []
    
    for key, pred_df in fold_predictions.items():
        _, mname, group = key
        if mname == model_name and group == "All Features":
            y_proba_all.extend(pred_df["y_proba"])

    sns.kdeplot(y_proba_all, label=model_name, fill=True, alpha=0.3)

plt.title("Distribution of Predicted Probabilities")
plt.xlabel("p(left wins)")
plt.legend()
plt.savefig(IMAGE_DIR / "predicted_probabilities_hist.png", dpi=RESOLUTION)
plt.close()


## Feature group comparison

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6), sharex=False)

# --- Accuracy ---
sns.barplot(
    data=ablation_study_results,
    x="group",
    y="accuracy",
    errorbar="sd",
    ax=axes[0]
)
axes[0].set_title("Ablation Study: Accuracy by Feature Group")
axes[0].set_ylabel("Accuracy")
axes[0].set_xlabel("Feature Group")
axes[0].tick_params(axis='x', rotation=45)

# --- AUC ---
sns.barplot(
    data=ablation_study_results,
    x="group",
    y="auc",
    errorbar="sd",
    ax=axes[1]
)
axes[1].set_title("Ablation Study: AUC by Feature Group")
axes[1].set_ylabel("AUC")
axes[1].set_xlabel("Feature Group")
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig(IMAGE_DIR / "feature_group_stats.png", dpi=RESOLUTION)
plt.close()


## Individual feature importance

In [ ]:
all_importances = []

for group_name, cols in feature_groups.items():
    X_full = engineered_df[cols]
    model = deepcopy(best_model)
    model.fit(X_full, y)
    
    df_imp = pd.DataFrame({
        "group": group_name,
        "feature": cols,
        "importance": model.feature_importances_
    })
    all_importances.append(df_imp)

importance_df = pd.concat(all_importances, ignore_index=True)

In [ ]:
heatmap_df = importance_df.pivot_table(
    index="feature", columns="group", values="importance", fill_value=0
)

plt.figure(figsize=(12,heatmap_df.shape[0]/5))
sns.heatmap(heatmap_df, cmap="viridis", linewidths=0.5)
plt.title("Feature Importance Heatmap Across Groups")
plt.xlabel("Feature Group")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "feature_importance.png", dpi=RESOLUTION)
plt.close()

In [ ]:
global_ranking = importance_df.groupby("feature")["importance"].mean().sort_values(ascending=False).head(20)
plt.figure(figsize=(8,6))
sns.barplot(x=global_ranking.values, y=global_ranking.index, hue=global_ranking.index, palette="viridis")
plt.title("Top 20 Features Overall")
plt.xlabel("Mean Importance")
plt.ylabel("Feature")
plt.tight_layout()
plt.savefig(IMAGE_DIR / "top_20_features.png", dpi=RESOLUTION)
plt.close()